In [1]:
import sys
sys.path.append("..")
from TextToSolver import TextToSolver as tts
import numpy as np

from bokeh.io import curdoc, output_notebook, push_notebook
from bokeh.layouts import row, widgetbox
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import *
from ipywidgets import interact

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
residual = ['-pK1 + pCO3 + 2 * pH - pCO2',
            '-pK2 + pCO3 + pH - pHCO3',
            '-pKw + pH + pOH',
            '-log(10 ** pCO2T) + log(10 ** pCO2 + 10 ** pHCO3 + 10 ** pCO3)']

ind_vars = ['pH', 'pCO2T']
dep_vars = ['pHCO3', 'pCO3', 'pCO2', 'pOH']
parameters = {'pKw': -14, 'pK1': -16.681, 'pK2': -10.33}
options = {'DISPLAY': False}

system = tts(dep_vars, residual, indep_vars=ind_vars,
             parameters=parameters)

In [4]:
n = 100
pH = np.linspace(-14, 0, n)
pco2t = -3
ind_var = {'pH': pH, 'pCO2T': pco2t*np.ones(pH.shape)}

guess = {var: np.ones(pH.shape) for var in dep_vars}

solution, report = system.solve(guess, indep_var_val=ind_var,
                                input_options=options)
solution['pH'] = pH

source = ColumnDataSource(data=solution)

In [5]:
plot = figure(plot_height=400, plot_width=600, title="carbonate speciation", x_range=[-14, 0])
colors = ['red', 'cyan', 'green', 'blue']
for ind, var in enumerate(dep_vars):
    plot.line('pH', var, source=source, line_color=colors[ind], line_width=3,
              line_alpha=0.6)

plot.xaxis.axis_label = 'log10 [H^+]'
plot.yaxis.axis_label = 'log10 concentration'

In [6]:
def update_data(CO2T):

    # Get the current slider values
    ind_var_val = {'pH': pH, 'pCO2T': CO2T*np.ones(pH.shape)}
    new_sol, report = system.solve(guess, indep_var_val=ind_var_val,
                           input_options=options)
    new_sol['pH'] = pH
    source.data = new_sol
    push_notebook()

In [7]:
# show the plot
show(plot, notebook_handle=True)

In [8]:
interact(update_data, CO2T = (-5, 5))

A Jupyter Widget

<function __main__.update_data>